In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import neologdn
import time
import os
import urllib.request
from matplotlib.font_manager import FontProperties
import MeCab
from __future__ import unicode_literals
import re
import unicodedata
import sagemaker
from sagemaker import get_execution_role
import boto3
import json
import pandas as pd
import warnings
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import codecs
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

#chaneg dummies to matrix
from scipy.sparse import csr_matrix,coo_matrix

import multiprocessing

from multiprocessing import Pool

import pickle

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from keras import backend as K
from keras.models import Model
from keras import initializers
from keras.engine.topology import Layer
from keras.layers import Dense, Input
from keras.layers import Embedding, GRU, Bidirectional, TimeDistributed,CuDNNGRU,CuDNNLSTM
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.utils.np_utils import to_categorical
from nltk import tokenize
import os
import tarfile
import numpy as np

import string

import re
import nltk



import gensim
path="/home/ec2-user/SageMaker/Notebooks_For_CX_Usecases/Multiple_Labels_Text_Classfication/notebook/models/word2vec-models/word2vec_100d_mfreq5.model"
word_vectors = gensim.models.Word2Vec.load(path)

In [ ]:
#tokenize japanase words
def wakati_by_mecab(text):
    tagger = MeCab.Tagger('-Ochasen -d /usr/lib64/mecab/dic/mecab-ipadic-neologd')
    tagger.parse('') 
    node = tagger.parseToNode(text)
    word_list = []
    while node:
        pos = node.feature.split(",")[0]
        #if pos in ["名詞","動詞","形容詞","記号"]:   # 対象とする品詞
        if pos in ["名詞"]:   # 対象とする品詞
            word = node.surface
            word_list.append(word)
        node = node.next
    return " ".join(word_list)

In [ ]:
def remove_chars(row):
    cust_list=['\d+','■','\n','・','◇','①','②','③','④','【】','】','【','⇒','●','★',':','※','→']
    del_list = string.ascii_letters + string.punctuation
    for i in del_list :
        row = row.str.replace(i,'')
    for i in cust_list :
        row = row.str.replace(i,'')
    return row

In [ ]:
file_path="/home/ec2-user/SageMaker/Notebooks_For_CX_Usecases/Multiple_Labels_Text_Classfication/raw_data/raw_claim_data.xlsx"
df = pd.read_excel(file_path, sheetname=None, ignore_index=True)

In [ ]:
df_sheet_all = pd.concat(df.values())

In [ ]:
df_sheet_all.reset_index(drop=True,inplace=True)

In [ ]:
cols=['協会分類原因'
     ,'苦情の要約（100文字以内程度）','申出内容詳細1（500文字以内程度）','申出内容詳細2（500文字以内程度）']

In [ ]:
#cols=['協会分類原因','協会分類（大）','協会分類（中）','協会分類（小Ⅰ）','協会分類（小Ⅱ）'
#     ,'苦情の要約（100文字以内程度）']

In [ ]:
df_col=df_sheet_all[cols]
df_col.columns = ['kyokai_reason','cmt0','cmt1','cmt2']

In [ ]:
#df_col=df_sheet_all[cols]
#df_col.columns = ['kyokai_reason','kyokai_big','kyokai_mid','kyokai_big_small_1', 'kyokai_big_small_2','cmt0']

In [ ]:
#削除いらない分類
#indexNames = df_col[ (df_col['kyokai_big_small_1'] == "022 営業店からの請求書類送付遅い") | (df_col['kyokai_big_small_1'] == "書類到着確認フォロー") ].index
#df_col.drop(indexNames, inplace=True)

In [ ]:
# replace old with new values
#df_col.replace({'kyokai_big_small_2': {"012 担当者訪問なし": "012 アフターフォロー不足", 
#                                       "013 保全等手続き･説明不十分": "013 説明不十分、対応不満",
#                                       "034 保険の仕組（例、掛捨、満期）":"保険の仕組（例、掛捨、満期、変額）"}},inplace=True)

#df_col.replace({'claim_B': {"001 ａ：通知類が見づらい": "001 ａ：通知類圧着不具合", 
#                                       "003 ｃ：通知類の情報量が足りない": "003 ｃ：通知類が分かりにくい、情報が足りない",
#                                       "007 ｇ：対応不十分（CSC）":"007 ｇ：対応不十分（CSC）・オペリスク"}},inplace=True)

In [ ]:
df_col.fillna("",inplace=True)

In [ ]:
def remove_non_num(s):
    return re.sub('[^0-9]+', '', s) 

In [ ]:
#only keep numbers in category
for col in ['kyokai_reason']:
    df_col[col]=df_col[col].apply(remove_non_num)

df_tra=df_col.copy()

In [ ]:
#pd.get_dummies(df_col, drop_first=True, prefix=['kyokai_reason'], prefix_sep='-')

cols_to_transform=['kyokai_reason']
df_tra=pd.get_dummies(df_col,columns=cols_to_transform)

In [ ]:
#combine text
cols = ['cmt0','cmt1','cmt2']
df_tra['comment'] =df_tra[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df_tra.drop(columns=cols,inplace=True)

In [ ]:
df_tra['comment'] = remove_chars(df_tra['comment'])
df_tra['comment']=df_tra['comment'].apply(neologdn.normalize)

In [ ]:
def return_sent(comment):
    sent_detector = nltk.RegexpTokenizer(u'[^　！？。]*[！？。.\n]')
    sents = sent_detector.tokenize(comment)
    return sents

In [ ]:
maxlen = 100
max_sentences = 20
max_words = 20000
embedding_dim = 300
validation_split = 0.3
reviews = []
labels = []
texts = []
glove_dir = "./glove.6B"
embeddings_index = {}

In [ ]:
for idx in range(df_tra.comment.shape[0]):
    text = df_tra.comment[idx]
    texts.append(text)
    sentences = return_sent(text)
    reviews.append(sentences)

labels=df_tra[df_tra.columns[:-1]].values

In [ ]:
pool = Pool(processes=multiprocessing.cpu_count())
df_tra['comment'] = pool.map(wakati_by_mecab,df_tra['comment'])
pool.close() 
pool.join()

In [ ]:
tokenized_text=list(df_tra['comment'].values)
tokenizer = Tokenizer(num_words=max_words,lower=False)
tokenizer.fit_on_texts(tokenized_text)

data = np.zeros((len(texts), max_sentences, maxlen), dtype='int32')

for i, sentences in enumerate(reviews):
    for j, sent in enumerate(sentences):
        if j < max_sentences:
            wordTokens = wakati_by_mecab(sent).split()
            k = 0
            for _, word in enumerate(wordTokens):
                try:
                    if k < maxlen and tokenizer.word_index[word] < max_words:
                        data[i, j, k] = tokenizer.word_index[word]
                        k = k + 1
                except:
                    data[i, j, k]=20001
                    k = k + 1

with open('sent_word_arr.pkl','wb') as f:
    pickle.dump(data, f)

In [ ]:
with open('sent_word_arr.pkl','rb') as f:
    data = pickle.load(f)

In [ ]:
word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

In [ ]:
print('Shape of reviews (data) tensor:', data.shape)
print('Shape of sentiment (label) tensor:', labels.shape)

In [ ]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(validation_split * data.shape[0])

In [ ]:
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [ ]:
print('Number of each category in training and validation set')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

In [ ]:
def load_embeddings(path):
    with open(path,'rb') as f:
        emb_arr = pickle.load(f)
    return emb_arr

In [ ]:
tic = time.time
fasttext_embeddings=load_embeddings('/home/ec2-user/SageMaker/word2vector/fasttext_300d.pkl')
print('loaded '+ str(len(fasttext_embeddings)) +' word vectors in {time.time()-tic}s')

In [ ]:
print('Total %s word vectors.' % len(fasttext_embeddings))

In [ ]:
# building Hierachical Attention network
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    try:
        embedding_vector = fasttext_embeddings[word]
        embedding_matrix[i] = embedding_vector
        
    except:
        # words not found in embedding index will be all-zeros.
        continue
        


In [ ]:
from keras import initializers as initializers, regularizers, constraints

In [ ]:
class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [ ]:
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

In [ ]:
embedding_layer = Embedding(len(word_index) + 1, embedding_dim, weights=[embedding_matrix],
                            input_length=maxlen, trainable=True, mask_zero=True)

In [ ]:
char_input = Input(shape=(maxlen,), dtype='int32')
char_sequences = embedding_layer(char_input)
char_lstm = Bidirectional(GRU(100, return_sequences=True))(char_sequences)
char_dense = TimeDistributed(Dense(200))(char_lstm)
char_att = AttentionWithContext()(char_dense)
charEncoder = Model(char_input, char_att)

words_input = Input(shape=(max_sentences, maxlen), dtype='int32')
words_encoder = TimeDistributed(charEncoder)(words_input)
words_lstm = Bidirectional(GRU(100, return_sequences=True))(words_encoder)
words_dense = TimeDistributed(Dense(200))(words_lstm)
words_att = AttentionWithContext()(words_dense)
preds = Dense(5, activation='softmax')(words_att)
model = Model(words_input, preds)

In [ ]:
from keras.callbacks import Callback
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [ ]:
RocAuc = RocAucEvaluation(validation_data=(x_val, y_val), interval=1)
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=5, batch_size=600)

In [ ]:
df=pd.read_pickle('processed.pkl')

In [ ]:
df_tra['comment']=df['comment']

In [ ]:
df_tra['comment'] = remove_chars(df_tra['comment'])
df_tra['comment']=df_tra['comment'].apply(neologdn.normalize)
pool = Pool(processes=multiprocessing.cpu_count())
df_tra['comment'] = pool.map(wakati_by_mecab,df_tra['comment'])
pool.close() 
pool.join()

In [ ]:
df_tra['comment'].head(10)

In [ ]:
def return_len(cmt):
    return len(cmt)

In [ ]:
#df_tra['comment'].apply(return_len).sort_values(ascending=False)
#df_tra['len']

In [ ]:
from sklearn.model_selection import train_test_split

#train, test = train_test_split(df_mutilabel, test_size=0.2)

train_texts = df_tra["comment"].values
train_labels = df_tra[df_tra.columns[:-1]].values

#test_texts = test["comment"].values
#test_labels = test[['cat_brochure', 'cat_compass', 'cat_myaxa', 'cat_reputaion', 'cat_keiyaku','cat_tetuduki','cat_tanto','cat_other']].values



In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

text=list(df_tra['comment'].values)


top_vocab=10000

tokenizer = Tokenizer(top_vocab)
#list xは上に書いたようにテキストデータが格納されている
tokenizer.fit_on_texts(text)

maxlen_value=1000

# 对每一句影评文字转换为数字列表，使用每个词的编号进行编号
x_train_seq = tokenizer.texts_to_sequences(train_texts)
#x_test_seq = tokenizer.texts_to_sequences(test_texts)
x_train = sequence.pad_sequences(x_train_seq, maxlen=maxlen_value)
#x_test = sequence.pad_sequences(x_test_seq, maxlen=maxlen_value)
y_train = np.array(train_labels)
#y_test = np.array(test_labels)
with open('tokenizer/kyokai_reason_ML_TXT_Claim_tokenizer_BiLSTM_BiGRU.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import copy
x = list(tokenizer.word_counts.items())
s = sorted(x,key=lambda p:p[1],reverse=True)
word_index = tokenizer.word_index # 不受vocab_size的影响
small_word_index = copy.deepcopy(word_index) # 防止原来的字典也被改变了
print("Removing less freq words from word-index dict...")
for item in s[top_vocab:]:
    small_word_index.pop(item[0])
print("Finished!")
print(len(small_word_index))
print(len(word_index))

In [ ]:
import gensim
#配布されていたfastTextの次元数は300だったので300
EMBEDDING_DIM = 100
#embedding_metrix = np.zeros(top_vocab+1,EMBEDDING_DIM)
embedding_matrix = np.random.uniform(size=(top_vocab+1,EMBEDDING_DIM))
#word_vectors = gensim.models.KeyedVectors.load_word2vec_format('/home/ec2-user/SageMaker/entitiy_vector_dic/entity_vector/entity_vector.model.bin', binary=True)
for word, i in small_word_index.items():
    try:
        #print (word)
        embedding_vector = word_vectors.wv[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        print("Word: [",word,"] not in wvmodel! Use random embedding instead.")

In [ ]:
def full_multi_label_metric(y_true, y_pred):
    comp = K.equal(y_true, K.round(y_pred))
    return K.cast(K.all(comp, axis=-1), K.floatx())

In [ ]:
import tensorflow as tf
from keras import backend as K
from keras.models import Model
from keras.layers import Dense, Embedding, Input, concatenate, Flatten, SpatialDropout1D
from keras.layers import CuDNNLSTM,LSTM, Bidirectional, GlobalMaxPool1D, Dropout, CuDNNGRU,GRU, GlobalAveragePooling1D, GlobalMaxPooling1D,Conv1D
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard, LearningRateScheduler, Callback
from keras.optimizers import Adam, Adadelta, SGD, RMSprop, Nadam

def get_model():
    inputs = Input(shape=(maxlen_value,))
    class_count = 5
    wv_dim=100
    vocab_size=top_vocab
    use_pretrained_wv = True
    if use_pretrained_wv:
        wv = Embedding(vocab_size+1,wv_dim,input_length=maxlen_value,weights=[embedding_matrix],trainable=True)(inputs)
    else:
        wv = Embedding(vocab_size+1,wv_dim,input_length=MAXLEN)(inputs)
        
    #x = SpatialDropout1D(0.2)(wv)
    x = Bidirectional(CuDNNGRU(256, return_sequences=True))(wv)
    x = Bidirectional(CuDNNGRU(256, return_sequences=True))(x)
    x = Conv1D(64,8,padding='same', activation='relu')(x)
    x = Dropout(0.2)(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool])
    x = Dense(128, activation="relu")(x)
    x = Dense(class_count, activation="softmax")(x)
    model = Model(inputs=inputs, outputs=x)
    model.summary()
    opt = Adam(lr=1e-3)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [ ]:
checkpoint_path = "models/kyoka_reason_ML_TXT_Claim_LSTM-GRU-Stack-weights-improvement-{val_acc:.4f}.hdf5"
checkpoint_dir = os.path.dirname(checkpoint_path)
 
cp_callback = ModelCheckpoint(checkpoint_path,
                              save_best_only=True,
                              verbose=0)
 
callbacks = [
    EarlyStopping(patience=150, monitor='val_acc',mode='max'),
    cp_callback
]

In [ ]:
batch_size = 128
epochs = 10
m = get_model()

m.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,shuffle=True,validation_split=0.2,callbacks=callbacks,verbose=1)
